In [1]:
import pandas as pd, numpy as np

In [2]:
data = np.array([[24, 1355], [35, 603], [21, 192], [30, 224]])
data = np.append(data, (data[:,0]/(data[:,0] + data[:,1])).reshape(-1, 1), axis=1)
index = [0, 2, 4, 5]

In [3]:
df = pd.DataFrame(data = data, index = index, columns=['HD', 'No HD', 'HD Proportion'])
df

,HD,No HD,HD Proportion
0,24.0,1355.0,0.017404
2,35.0,603.0,0.054859
4,21.0,192.0,0.098592
5,30.0,224.0,0.118110


The Newton-Raphson method can be used to solve logistic regression in accordance with the expression found in (4.45) in the book, $\boldsymbol{\beta}^{(t+1)} = \boldsymbol{\beta}^{(t)} - \left(\boldsymbol{H}^{(t)}\right)^{-1}\boldsymbol{u}^{(t)}$.

For logistic regression, $Y_i \sim \text{Bernoulli}(\pi_i)$, where the link function is 
$$\log{\frac{\pi(x)}{1-\pi(x)}} =\alpha + \beta x$$ or
$$\pi(x) = \frac{\exp{\alpha + \beta x}}{1+\exp{\alpha + \beta x}}. $$


From the Bernoulli mass function $\pi^y(1-\pi)^{1-y}$, we derive the log-likelihood function for the parameters given $x_i$ and $y_i$, $i \in {1, ..., n}$ as 
\begin{align}
L(\pi|x_i, y_i) &= \prod_{i=1}^n \pi(x_i)^y_i(1-\pi(x_i))^{1-y_i} \\
&=\prod_{i=1}^n (1-\pi(x_i))\exp{\left(y\log{\frac{\pi(x_i)}{1-\pi(x_i)}}\right)} \\
\mathcal{L}(\pi|x_i, y_i) &= \sum_{i=1}^n \log{\left(1-\pi(x_i)\right)} + y_i\log{\left(\frac{\pi(x_i)}{1-\pi(x_i)}\right)} \\
\mathcal{L}(\alpha, \beta|x_i, y_i) &= \sum_{i=1}^n -\log{\left(1+\exp{\left(\alpha+\beta x_i\right)}\right)}+y_i(\alpha+\beta x_i)
\end{align}

There are two parameters in the $\boldsymbol{\beta}$ of the Newton-Raphson method, $\alpha$ and $\beta$. For these parameters, we derive the gradient vector $$\boldsymbol{u} = \left[\begin{matrix}\dfrac{\partial\mathcal{L}}{\partial\alpha}& \dfrac{\partial\mathcal{L}}{\partial\beta}\end{matrix}\right] = 
\left[\begin{matrix}\sum y_i - \dfrac{\exp{\left(\alpha+\beta x_i\right)}}{1+\exp{\left(\alpha+\beta x_i\right)}}&
\sum x_i\left(y_i - \dfrac{\exp{\left(\alpha+\beta x_i\right)}}{1+\exp{\left(\alpha+\beta x_i\right)}}\right)\end{matrix}\right]$$
and the Hessian matrix 
$$\boldsymbol{H} = \left[\begin{matrix}\dfrac{\partial^2\mathcal{L}}{\partial\alpha^2}&\dfrac{\partial^2\mathcal{L}}{\partial\alpha\partial\beta}\\ \dfrac{\partial^2\mathcal{L}}{\partial\alpha\partial\beta}&\dfrac{\partial^2\mathcal{L}}{\partial\beta^2}\end{matrix}\right] = \left[\begin{matrix}\sum\dfrac{\exp{\left(\alpha+\beta x_i\right)}}{[1+\exp{\left(\alpha+\beta x_i\right)]^2}}&\sum\dfrac{x_i\exp{\left(\alpha+\beta x_i\right)}}{[1+\exp{\left(\alpha+\beta x_i\right)]^2}}\\ \sum\dfrac{x_i\exp{\left(\alpha+\beta x_i\right)}}{[1+\exp{\left(\alpha+\beta x_i\right)]^2}}&\sum\dfrac{x_i^2\exp{\left(\alpha+\beta x_i\right)}}{[1+\exp{\left(\alpha+\beta x_i\right)]^2}}\end{matrix}\right]$$

In [149]:
def pi_x(param, x):
    alpha = param[0]; beta = param[1]
    return np.exp(alpha + beta * x)/(1 + np.exp(alpha + beta * x))

In [150]:
def gradient(param, x, y): 
    alpha = param[0]; beta = param[1] # specifically for a length-two parameter vector
    pr = np.exp(alpha + beta * x)/(1+np.exp(alpha + beta * x))
    dLda = y - pi_x(param, x)
    dLdb = x*(y - pi_x(param, x))
    return np.array([sum(dLda), sum(dLdb)])

In [151]:
def hessian(param, x, y):
    alpha = param[0]; beta = param[1]
    pr = np.exp(alpha + beta * x)/(1+np.exp(alpha + beta * x))
    dLdaa = pi_x(param, x)*(1-pi_x(param, x))
    dLdab = x*pi_x(param, x)*(1-pi_x(param, x))
    dLdbb = x**2*pi_x(param, x)*(1-pi_x(param, x))
    return np.array([[sum(dLdaa), sum(dLdab)], [sum(dLdab), sum(dLdbb)]])

Now we will initialize our variables to implement the Newton-Raphson solver. The independant variable ($x$) is the index value of the snoring categories of our table. The dependent variable ($y$) is the proportion of people with heart disease.

In [172]:
x = np.array(index)
y = df['HD Proportion']
param = [0.1, 0.1] # initial guesses

In [171]:
dparam = np.dot(np.linalg.inv(hessian(param, x, y)), gradient(param, x, y))
param = param + dparam
dparam, param

(array([-2.02818784, -0.05298974]), array([-1.92818784,  0.04701026]))

In [173]:
delta = 1e-10
iter_limit = 100
dp = np.Infinity
i=0
while dp > delta and i < iter_limit:
    dparam = np.dot(np.linalg.inv(hessian(param, x, y)), gradient(param, x, y))
    param = param - dparam
    dp = np.sum(dparam**2) # a function to measure how much we are changing each iteration
    i += 1
print("Iterated {0} times".format(i))

Iterated 4 times


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [142]:
param

array([ nan,  nan])

Our final parameters are $\alpha = -3.729$ and $\beta = 0.359$. We can now add our estimated fit values to the table

In [128]:
df['Logistic Fit'] = pi_x(param, x)
df

,HD,No HD,HD Proportion,Logistic Fit
0,24.0,1355.0,0.017404,0.023463
2,35.0,603.0,0.054859,0.046992
4,21.0,192.0,0.098592,0.091896
5,30.0,224.0,0.118110,0.126614
